# Jaccard Similarity between Dependency Trees

In [1]:
import sys
sys.path.append("..")

In [2]:
import conllu
import treesimi as ts
import json
import datasketch

## Load dataset

In [3]:
%%capture
!mkdir "../data"
!wget -nc -O "../data/de_hdt-ud-dev.conllu" "https://raw.githubusercontent.com/UniversalDependencies/UD_German-HDT/master/de_hdt-ud-dev.conllu"

In [4]:
dat = conllu.parse(open("../data/de_hdt-ud-dev.conllu").read())

## 32-bit mmh3 Hashes

In [5]:
import struct
import mmh3

def hashfunc_mmh3_int32(data: bytes):  # -> packed bytes unsigned 32-bit
    """ Hash function for DataSketch
    Example:
    --------
    txt = "Hello World!"
    b = hashfunc_mmh3_int32(txt.encode('utf-8'))
    """
    return struct.unpack('<I', struct.pack('<l', mmh3.hash(data)))[0]

_hashfunc = hashfunc_mmh3_int32
# _hashfunc = datasketch.hashfunc.sha1_hash32

## Generate shinglesets

In [6]:
cfg = {
    'use_trunc_leaves': True, 
    'use_drop_nodes': False, 
    'use_replace_attr': False
}

texts = []
mhash = []
for i in (54, 51, 56, 57, 58):
    adjac = [(t['id'], t['head'], t['deprel']) for t in dat[i]]
    nested = ts.adjac_to_nested_with_attr(adjac)
    nested = ts.remove_node_ids(nested)
    shingled = ts.shingleset(nested, **cfg)
    stringified = [json.dumps(tree).encode('utf-8') for tree in shingled]
    m = datasketch.MinHash(num_perm=32, hashfunc=_hashfunc)
    for s in stringified:
        m.update(s)
    mhash.append(m)
    # for printout lateron
    texts.append(dat[i].metadata.get("text"))


## Storable Hashes

In [7]:
import numpy as np

m = mhash[1]
hv = np.uint32(m.hashvalues)

print(f"{8 * hv.nbytes}-bit")
print(f"n_elems: {hv.size}")

hv2 = [ts.uint32_to_int32(i) for i in m.hashvalues]
min(hv2), max(hv2)

1024-bit
n_elems: 32


(1457604, 1489630792)

## Compute Jaccard Similarities

In [8]:
for i in range(len(mhash)):
    print(mhash[0].jaccard(mhash[i]), texts[i][:70])

1.0 Das Geschäft von Amazon sei der Buchhandel und nicht der Verkauf von K
0.09375 Amazon.com möchte sich nun offensichtlich den Rücken für den Verkauf s
0.15625 Allerdings steckt Amazon.com tief in den roten Zahlen ; die Verluste b
0.21875 Die deutsche Tochter hat ihre Datenschutzrichtlinien nicht wie die US-
0.09375 Die Richtlinien sehen jedoch vor , dass die in Deutschland erfassten D


In [9]:
for i in range(len(mhash)):
    print(np.mean(mhash[0].hashvalues == mhash[i].hashvalues), texts[i][:70])

1.0 Das Geschäft von Amazon sei der Buchhandel und nicht der Verkauf von K
0.09375 Amazon.com möchte sich nun offensichtlich den Rücken für den Verkauf s
0.15625 Allerdings steckt Amazon.com tief in den roten Zahlen ; die Verluste b
0.21875 Die deutsche Tochter hat ihre Datenschutzrichtlinien nicht wie die US-
0.09375 Die Richtlinien sehen jedoch vor , dass die in Deutschland erfassten D
